# TEPEDELEN Léo DELLARICA Steven BARBIN Kevin

# Projet EISI1 Capgemini MSPRTPRE813_1

Ce notebook Python est le rapport détaillé et l'ETL de notre projet. La suite du rapport se situe après l'ETL, nous y détaillons le fonctionnement du reste du système.

Il a été réalisé à l'aide de Docker afin d'en faciliter le déploiement pour tester et collaborer.

Vous trouverez tout ce qu'il vous faut pour essayer le programme sur votre machine dans le fichier README de ce repository github :

# Extraire les données dans un DataFrame pySpark

In [21]:
import pandas as pd

# candidats_results.parquet
df_election = pd.read_parquet('https://www.dropbox.com/scl/fi/04jpbz2gqcax0y8ubetz8/candidats_results.parquet?rlkey=cvs3816ot12wxg41ss02q4aag&st=0r4ilpa8&dl=1')

In [22]:
def fillNuance(df, col, value, nuance):
    df.loc[df[col] == value, ["Nuance"]] = nuance

In [23]:
missing_nuances_libelle = {
    "ALLIANCE JAUNE":"EGAUCHE",
    "ALLONS ENFANTS":"GAUCHE",
    "DEBOUT LA FRANCE":"EDROITE",
    "DÉCROISSANCE 2019":"EGAUCHE",
    "DÉMOCRATIE REPRÉSENTATIVE":"EGAUCHE",
    "ENSEMBLE PATRIOTES":"EDROITE",
    "ENSEMBLE POUR LE FREXIT":"EDROITE",
    "ENVIE D'EUROPE":"GAUCHE",
    "ESPERANTO":"EGAUCHE",
    "EUROPE AU SERVICE PEUPLES":"GAUCHE",
    "EUROPE ÉCOLOGIE":"GAUCHE",
    "INITIATIVE CITOYENNE":"EGAUCHE",
    "LA FRANCE INSOUMISE":"GAUCHE",
    "LA LIGNE CLAIRE":"EDROITE",
    "LES EUROPÉENS":"CENTRE",
    "LES OUBLIES DE L'EUROPE":"CENTRE",
    "LISTE CITOYENNE":"GAUCHE",
    "LISTE DE LA RECONQUÊTE":"EDROITE",
    "LUTTE OUVRIÈRE":"EGAUCHE",
    "NEUTRE ET ACTIF":"EDROITE",
    "PACE":"GAUCHE",
    "PARTI ANIMALISTE":"CENTRE",
    "PARTI FED. EUROPÉEN":"EDROITE",
    "PARTI PIRATE":"GAUCHE",
    "POUR L'EUROPE DES GENS":"EGAUCHE",
    "PRENEZ LE POUVOIR":"EDROITE",
    "RENAISSANCE":"CENTRE",
    "RÉVOLUTIONNAIRE":"EGAUCHE",
    "UDLEF":"DROITE",
    "UNE FRANCE ROYALE":"EDROITE",
    "UNION DROITE-CENTRE":"CDROIT",
    "URGENCE ÉCOLOGIE":"EGAUCHE",
    "À VOIX ÉGALES":"GAUCHE",
    "ÉVOLUTION CITOYENNE":"GAUCHE"
}

In [24]:
missing_nuances_nom = {
    "ARTHAUD":"EGAUCHE",
    "ASSELINEAU":"EDROITE",
    "CHEMINADE":"EDROITE",
    "DUPONT-AIGNAN":"EDROITE",
    "FILLON":"DROITE",
    "HAMON":"GAUCHE",
    "HIDALGO":"GAUCHE",
    "JADOT":"GAUCHE",
    "LASSALLE":"DROITE",
    "LE PEN":"EDROITE",
    "MACRON":"CENTRE",
    "MÉLENCHON":"GAUCHE",
    "POUTOU":"EGAUCHE",
    "PÉCRESSE":"DROITE",
    "ROUSSEL":"EGAUCHE",
    "ZEMMOUR":"EDROITE"
}

In [25]:
for libelle, nuance in missing_nuances_libelle.items():
    fillNuance(df_election, "Libellé Abrégé Liste", libelle, nuance)

for nom, nuance in missing_nuances_nom.items():
    fillNuance(df_election, "Nom", nom, nuance)

In [26]:
import urllib.request
import json
with urllib.request.urlopen('https://www.dropbox.com/scl/fi/trxktezbq12phyqptj545/nuances.json?rlkey=5u4368x6evhct81slh20ffzii&st=mw31zc2i&dl=1') as outfile:
     dic_nuances = json.load(outfile)

In [27]:
def replaceValue(nom_nuance, dic_nuances):
    for nuance, list_nom_nuance in dic_nuances.items():
        if nom_nuance in list_nom_nuance:
            return nuance
    return nom_nuance

df_election['Nuance'] = df_election['Nuance'].apply(lambda nom_nuance: replaceValue(nom_nuance, dic_nuances))

In [28]:
df_election = df_election[~df_election["id_election"].str.contains(r'muni|dpmt|regi|cant', na=False)]

In [29]:
df_election = df_election[["id_election","Code du département","Code de la commune", "Code du b.vote", "% Voix/Ins", "Nuance"]]

In [30]:
df_election[["annee", "type_election", "tour"]] = df_election["id_election"].str.split("_", expand=True)

In [31]:
# df_election["Code de la commune"] = df_election.apply(lambda row: row["Code du département"] + row["Code de la commune"], axis=1)
df_election['Code de la commune'] = df_election['Code du département'] + df_election['Code de la commune']

In [33]:
df_election["Code de la commune"] = pd.to_numeric(df_election["Code de la commune"], errors='coerce')

In [34]:
df_election.dropna(subset=["Code du département", "Code de la commune"], inplace=True)
df_election["Code de la commune"] = df_election["Code de la commune"].astype(int)

In [35]:
df_election.rename(columns={
    "Code du département":"dep",
    "Code de la commune":"codecommune",
    "Code du b.vote":"bureau",
    "% Voix/Ins":"pourcentage_voix_inscrits",
    "Nuance":"nuance"
}, inplace=True)

In [36]:
df_election = df_election[["id_election", "codecommune", "bureau", "annee", "type_election", "tour", "nuance", "pourcentage_voix_inscrits"]].groupby(["id_election", "codecommune", "bureau", "annee", "type_election", "tour", "nuance"]).sum()

In [37]:
df_election.reset_index(inplace=True)

In [38]:
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder.appName("example") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "1") \
    .config("spark.dynamicAllocation.maxExecutors", "50") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.memory.fraction", "0.8") \
    .getOrCreate()

In [39]:
df_election = spark.createDataFrame(df_election)

In [40]:
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

df_election.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="elections", mode="overwrite", properties=properties)

In [ ]:
df_election.unpersist()
del df_election
del missing_nuances_nom
del missing_nuances_libelle

In [ ]:
# popcommunes.csv
df_demographie = pd.read_csv('https://www.dropbox.com/scl/fi/qayze0v5xxwolw4tj8qmc/popcommunes.csv.zip?rlkey=qnpfvfq0kc096dykwfvz2jrv0&st=gk60a3ff&dl=1', compression='zip', header=0, sep=',', quotechar='"')

# cspcommunes.csv
df_csp = pd.read_csv('https://www.dropbox.com/scl/fi/opg91qcti1ctwmopr4efj/cspcommunes.csv.zip?rlkey=h8pti3tzoo7tqjctc2h1rfeqx&st=1mnddf1h&dl=1', compression='zip', header=0, sep=',', quotechar='"')

# df_criminalite = pd.read_csv('https://drive.google.com/uc?export=download&id=1F9Jm0UaemicMy4MnvZ47nmZHiIdsmRwC')

In [ ]:
# Using pandas

df_demographie_filtre = df_demographie.filter(regex="^(dep|nomdep|codecommune|nomcommune|reg|nomreg)$|^pop[0-9]{4}$")

df_csp_filtre = df_csp.filter(regex="^(dep|nomdep|codecommune|nomcommune|agri|indp|cadr|pint|empl|ouvr|chom|pact|chom)(\d{4})?$")

In [ ]:
# # 1. Create the dimension table for commune data (df_dimension_commune)
# df_dimension_commune = df_demographie_filtre[['dep', 'codecommune', 'nomcommune', 'reg']].drop_duplicates()

# # 2. Extract population columns (columns starting with "pop" followed by 4 digits)
# pop_columns = [col for col in df_demographie_filtre.columns if col.startswith('pop') and len(col) == 7 and col[3:].isdigit()]

# # 3. Pivot the population columns into long format for the facts table (df_faits)
# # We use the `melt` function to transform wide to long format
# df_faits = pd.melt(df_demographie_filtre, 
#                    id_vars=['codecommune'],  # Keep 'codecommune' as ID column
#                    value_vars=pop_columns,   # These are the population columns to melt
#                    var_name='annee',          # Name of the new 'annee' column
#                    value_name='population')  # Name of the new 'population' column

# # Extract annee from 'popXXXX' column
# df_faits['annee'] = df_faits['annee'].str[-4:]

# print("Dimension Commune Table:")
# print(df_dimension_commune)

# print("\nFacts Table (Population Data):")
# print(df_faits)

In [ ]:
# Using pySpark

df_demographie_filtre = spark.createDataFrame(df_demographie_filtre)

In [ ]:
df_csp_filtre = spark.createDataFrame(df_csp_filtre)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
# # Using pySpark


# # 1. Create the dimension table for commune data (df_dimension_commune)
# df_dimension_commune = df_demographie_filtre.select("dep", "codecommune", 'nomcommune', "reg").distinct()

# # 2. Extract population columns (columns starting with "pop" followed by 4 digits)
# pop_columns = [col for col in df_demographie_filtre.columns if col.startswith('pop') and len(col) == 7 and col[3:].isdigit()]

# # 3. Pivot the population columns into long format for the facts table (df_faits)
# df_faits = df_demographie_filtre \
#     .withColumn("annee_population", F.explode(F.array(
#         *[F.struct(F.lit(col[-4:]).alias("annee"), F.col(col).alias("population")) for col in pop_columns]
#     )))

# # Split the "annee_population" struct into separate columns for annee and population
# df_faits = df_faits.withColumn("annee", F.col("annee_population.annee")) \
#                    .withColumn("population", F.col("annee_population.population")) \
#                    .drop("annee_population")

# # 4. Join the facts table with the commune table to include commune_id in df_faits
# df_faits = df_faits.select("codecommune", "annee", "population")

# # Display the results
# df_dimension_commune.show()
# df_faits.show()

In [ ]:
def getColumnsWithStringAndFourDigits(str, df):
    return [col for col in df.columns if col.startswith(str) and len(col) == len(str)+4 and col[-4:].isdigit()]

In [ ]:
# # popcommunes

# # 1. Optimize commune dimension table creation by dropping duplicates
# df_dimension_commune = df_demographie_filtre.select("dep", "codecommune", 'nomcommune', "reg").drop_duplicates()

# # 2. Extract population columns (columns starting with "pop" followed by 4 digits)
# pop_columns = getColumnsWithStringAndFourDigits("pop", df_demographie_filtre)

# # 3. Use stack() to pivot the population columns more efficiently for large datasets
# df_faits = df_demographie_filtre.select("codecommune", 
#                                         F.expr("stack({0}, {1})".format(
#                                             len(pop_columns),
#                                             ','.join([f"'{col[-4:]}', {col}" for col in pop_columns])
#                                         )).alias("annee", "population"))

# # Display the results
# # df_dimension_commune.show()
# # df_faits.show()

In [ ]:
# cspcommunes

from collections import OrderedDict

# 1. Optimize commune dimension table creation by dropping duplicates
df_dimension_commune = df_demographie_filtre.select("dep","nomdep", "codecommune", 'nomcommune', "reg", "nomreg").join(
                        df_csp_filtre.select("dep","nomdep", "codecommune", 'nomcommune'), on=["dep", "codecommune", "nomdep", 'nomcommune'], how="outer").drop_duplicates().dropna(subset=["reg", "nomreg"]).cache()

# 2. Extract population columns (columns starting with "pop" followed by 4 digits)
pop_columns = getColumnsWithStringAndFourDigits("pop", df_demographie_filtre)

csp_columns = OrderedDict()
for str in ["agri","indp","cadr","pint","empl","ouvr","chom","pact", "chom"]:
    csp_columns[str] = getColumnsWithStringAndFourDigits(str, df_csp_filtre)

annees_csp = OrderedDict()
for key, col_list in csp_columns.items():
    for col_name in col_list:
        annees_csp.setdefault(col_name[-4:], []).append(col_name) 


In [ ]:

# 3. Use stack() to pivot the population columns more efficiently for large datasets
df_faits_population = df_demographie_filtre.select("codecommune", F.expr("stack({0}, {1})".format(
                                                        len(pop_columns), ','.join([f"'{col[-4:]}', {col}" for col in pop_columns])
                                        )).alias("annee", "population"))


cast="CAST("
as_double=" AS DOUBLE)"

df_faits_csp = df_csp_filtre.select("codecommune", 
                                F.expr("stack({0}, {1})".format(len(annees_csp),
                                ','.join([f"'{annee}', {', '.join([cast + s + as_double for s in col_name])}" for annee, col_name in annees_csp.items()])
                                )).alias("annee", "agriculteurs", "independants", "cadres", "intermediaires", "employes", "ouvriers", "population_active_totale", "chomeurs"))

# df_faits = df_faits_population.join(df_faits_csp, on=["codecommune", "annee"], how="outer")
df_faits = df_faits_csp.join(df_faits_population, on=["codecommune", "annee"], how="outer").cache()

# Display the results
print(df_dimension_commune.count())
print(df_faits.count())

In [ ]:
df_dimension_commune.show()

In [ ]:
df_faits = df_faits.na.replace(float("nan"), None)
df_faits.show()

In [ ]:
# ','.join([f"'{col[-4:]}', {col}" for col in pop_columns])

In [ ]:
# Tentative infructueuse en utilisant pandas

# df_faits = pd.DataFrame(columns=["population"])
# df_faits = spark.createDataFrame(df_faits)

# df_dimension_commune = pd.DataFrame(columns=["code_departement", "code_commune", "code_region"])
# df_dimension_commune = spark.createDataFrame(df_dimension_commune)

# df_dimension_annee = pd.DataFrame(columns=["annee"])
# df_dimension_annee = spark.createDataFrame(df_dimension_annee)

# for idx, row in df_demographie_filtre.iterrows():
#     df_dimension_commune.loc[len(df_dimension_commune)] = [row["dep"], row["codecommune"], row["reg"]]
#     for index, value in row.filter(regex="^pop[0-9]{4}$").items():
#         df_faits.loc[len(df_faits)] = [value]
#         df_dimension_annee.loc[len(df_dimension_annee)] = [index[-4:]]

# Écriture des données transformées en base de données avec le driver JDBC

In [ ]:
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

df_faits.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="faits", mode="overwrite", properties=properties)

In [ ]:
df_dimension_commune.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="communes", mode="overwrite", properties=properties)

# Générer une table au format CSV avec une description statistique du DataFrame

In [ ]:
# df.describe().toPandas().to_csv("describe_summary.csv", index=False)

# Effectuer un échantillonage du DataFrame pour en observer les colonnes qui peuvent sembler pertinentes

In [ ]:
# FRACTION_SIZE = 0.00001

In [ ]:
# columnsToSample = ['quantity', 'serving_size', 'serving_quantity', 'product_quantity']

In [ ]:
# samples_not_clean_df = df.select(columnsToSample).dropna(how='all').cache()

In [ ]:
# samples_df = samples_not_clean_df.sample(withReplacement=False, fraction=FRACTION_SIZE).cache()

In [ ]:
# samples_df.count()

In [ ]:
# samples_df.show()

# Extraire les valeurs uniques d'une colonne afin d'en confirmer la pertinence (colonne ingredients_analysis_tags en exemple)

In [ ]:
# def flatten_list(li):
#     flat_list = []
#     for row in li:
#         flat_list += row
#     return flat_list

# def make_list_unique(li):
#     return list(dict.fromkeys(li))

# def split_string_list_elements(li, sep):
#     return [x.split(sep) for x in li]

# def column_to_list(col):
#     return col.rdd.flatMap(lambda x: x).collect()

In [ ]:
# ingredients_analysis_list = make_list_unique(flatten_list(split_string_list_elements(column_to_list(df.select('ingredients_analysis_tags').dropna()), ",")))

In [ ]:
# ingredients_analysis_list

# Sélection des colonnes nécessaires après étude de celles-ci

In [ ]:
# kept_columns = ["code", "product_name", "product_quantity", "energy-kcal_100g", "fat_100g", "saturated-fat_100g", "monounsaturated-fat_100g", "polyunsaturated-fat_100g", "trans-fat_100g", 
#                 "carbohydrates_100g", "sugars_100g", "starch_100g", "fiber_100g", "proteins_100g", "allergens", "traces", "vitamin-a_100g", "vitamin-c_100g", "vitamin-d_100g",
#                 "vitamin-e_100g", "vitamin-k_100g", "vitamin-b1_100g", "vitamin-b2_100g", "vitamin-b6_100g", "vitamin-b9_100g", "vitamin-b12_100g", "calcium_100g",
#                 "iron_100g", "magnesium_100g", "potassium_100g", "zinc_100g", "food_groups_tags", "serving_size", "serving_quantity", "cholesterol_100g", "salt_100g", "glycemic-index_100g"]

In [ ]:
# df_kept_columns = df.select(kept_columns)

# Qualité des données

## On supprime les lignes qui ont moins de 15 colonnes très pertinentes non nulles

In [ ]:
# columns_to_check = ["code", "product_name", "product_quantity", "energy-kcal_100g", "fat_100g", "saturated-fat_100g", "monounsaturated-fat_100g",
#                     "polyunsaturated-fat_100g", "trans-fat_100g", "carbohydrates_100g", "sugars_100g", "starch_100g", 
#                     "fiber_100g", "proteins_100g", "allergens", "traces", "food_groups_tags", "serving_size", "serving_quantity"]

In [ ]:
# df_kept_columns = df_kept_columns.dropna(subset=columns_to_check, thresh=15)

## On supprime les lignes dupliquées et on garde le DF en cache pour éviter de le recalculer

In [ ]:
# df_kept_columns = df_kept_columns.dropDuplicates().cache()

In [ ]:
# print("Nombre de lignes restantes après le traitement : " + str(df_kept_columns.count()))

## Plus besoin du DataFrame de base donc on le retire de la mémoire cache

In [ ]:
# df.unpersist()
# print("Mémoire libérée")